<a href="https://colab.research.google.com/github/manuelcozar55/LexAIcon/blob/main/FTLexAIcon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?id=1FBozZB6s5pruTsnXp-SMuhqz4WmQoDIL)

###Fine Tunning un MLM para el proyecto LexAIcon
El primer paso es instalar e importar todo lo necesario.

In [1]:
!pip install transformers datasets bitsandbytes peft evaluate py wandb
import wandb
from google.colab import drive
import json
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

drive.mount('/content/drive')

# Inicializar wandb- API_KEY necesaria
wandb.init(project="FTLexAIcon_v0.1")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


A continuación se instancia el conjunto de datos para el fine tunning.

In [2]:
documents = {
  "document_types": {
    "multas": [
      {
        "title": "Multa - Ejemplo 1",
        "content": "MULTA POR INCUMPLIMIENTO DE NORMATIVA DE ESTACIONAMIENTO EN ZONA AZUL\n\nFecha de la Infracción: 15 de Mayo de 2024\n\nHora de la Infracción: 10:45 AM\n\nLugar de la Infracción: Calle Alfonso I, 125, Zaragoza, España\n\nNúmero de Expediente: ZA-2024-00567\n\nMatrícula del Vehículo: 1234ABC\n\nDescripción de la Infracción:\n\nEl vehículo con matrícula 1234ABC se encontraba estacionado en una zona de estacionamiento regulado (Zona Azul) sin exhibir el correspondiente ticket de pago visible en el parabrisas. Según la normativa vigente, es obligatorio adquirir y mostrar el ticket de estacionamiento en un lugar visible del vehículo.\n\nNormativa Aplicable:\n\nDe acuerdo con la Ordenanza Municipal de Estacionamiento Regulado de Zaragoza, artículo 5, sección 3, es obligatorio que todos los vehículos estacionados en la Zona Azul dispongan del ticket de estacionamiento correspondiente y que este sea claramente visible.\n\nImporte de la Multa:\n\nEl importe de la multa por esta infracción es de 60 euros.\n\nInstrucciones de Pago:\n\nEl pago de la multa debe realizarse en un plazo de 30 días naturales a partir de la fecha de la notificación de la multa. El pago puede realizarse de las siguientes maneras:\n- En línea a través de la página web del Ayuntamiento de Zaragoza.\n- En cualquier oficina de atención al ciudadano del Ayuntamiento.\n- Mediante transferencia bancaria a la cuenta indicada en la notificación recibida por correo.\n\nDerecho de Alegación:\n\nEl infractor tiene derecho a presentar alegaciones contra esta multa en un plazo de 15 días hábiles a partir de la fecha de la notificación. Las alegaciones deben presentarse por escrito y pueden ser entregadas en cualquiera de las oficinas de atención al ciudadano del Ayuntamiento o enviadas por correo postal a la dirección indicada en la notificación.\n\nNotificación:\n\nLa presente notificación se envía al titular del vehículo registrado en la Dirección General de Tráfico. En caso de que los datos del titular no estén actualizados, se ruega que se actualicen a la mayor brevedad posible para evitar futuras incidencias."
      },
      {
        "title": "Multa - Ejemplo 2",
        "content": "MULTA POR RUIDO EXCESIVO EN ZONA RESIDENCIAL\n\nFecha de la Infracción: 10 de Junio de 2024\n\nHora de la Infracción: 11:30 PM\n\nLugar de la Infracción: Calle San Miguel, 34, Zaragoza, España\n\nNúmero de Expediente: RZ-2024-00321\n\nDescripción de la Infracción:\n\nEl día 10 de junio de 2024, a las 11:30 PM, se detectó un ruido excesivo proveniente de la vivienda ubicada en Calle San Miguel, 34. El nivel de ruido excedía los límites permitidos según la normativa municipal, causando molestias a los vecinos y alterando el orden público. Se midió un nivel de ruido de 85 decibelios, superando el límite de 50 decibelios establecido para áreas residenciales durante horario nocturno.\n\nNormativa Aplicable:\n\nDe acuerdo con la Ordenanza Municipal de Ruidos y Vibraciones de Zaragoza, artículo 8, sección 2, el nivel máximo de ruido permitido en áreas residenciales durante el horario nocturno es de 50 decibelios.\n\nImporte de la Multa:\n\nEl importe de la multa por esta infracción es de 200 euros.\n\nInstrucciones de Pago:\n\nEl pago de la multa debe realizarse en un plazo de 30 días naturales a partir de la fecha de la notificación de la multa. El pago puede realizarse de las siguientes maneras:\n- En línea a través de la página web del Ayuntamiento de Zaragoza.\n- En cualquier oficina de atención al ciudadano del Ayuntamiento.\n- Mediante transferencia bancaria a la cuenta indicada en la notificación recibida por correo.\n\nDerecho de Alegación:\n\nEl infractor tiene derecho a presentar alegaciones contra esta multa en un plazo de 15 días hábiles a partir de la fecha de la notificación. Las alegaciones deben presentarse por escrito y pueden ser entregadas en cualquiera de las oficinas de atención al ciudadano del Ayuntamiento o enviadas por correo postal a la dirección indicada en la notificación.\n\nNotificación:\n\nLa presente notificación se envía al titular de la vivienda registrada en el Ayuntamiento de Zaragoza. En caso de que los datos del titular no estén actualizados, se ruega que se actualicen a la mayor brevedad posible para evitar futuras incidencias."
      },
      {
        "title": "Multa - Ejemplo 3",
        "content": "MULTA POR INCUMPLIMIENTO DE HORARIOS DE RECOGIDA DE BASURA\n\nFecha de la Infracción: 25 de Mayo de 2024\n\nHora de la Infracción: 8:30 AM\n\nLugar de la Infracción: Calle del Coso, 78, Zaragoza, España\n\nNúmero de Expediente: RB-2024-01234\n\nDescripción de la Infracción:\n\nEl día 25 de mayo de 2024, a las 8:30 AM, se detectó que la basura había sido depositada fuera del horario permitido en la Calle del Coso, 78. Según la normativa vigente, la basura debe depositarse en los contenedores entre las 8:00 PM y las 10:00 PM para evitar molestias y problemas de salubridad.\n\nNormativa Aplicable:\n\nDe acuerdo con la Ordenanza Municipal de Gestión de Residuos y Limpieza Urbana de Zaragoza, artículo 12, sección 5, es obligatorio respetar los horarios establecidos para el depósito de basura.\n\nImporte de la Multa:\n\nEl importe de la multa por esta infracción es de 100 euros.\n\nInstrucciones de Pago:\n\nEl pago de la multa debe realizarse en un plazo de 30 días naturales a partir de la fecha de la notificación de la multa. El pago puede realizarse de las siguientes maneras:\n- En línea a través de la página web del Ayuntamiento de Zaragoza.\n- En cualquier oficina de atención al ciudadano del Ayuntamiento.\n- Mediante transferencia bancaria a la cuenta indicada en la notificación recibida por correo.\n\nDerecho de Alegación:\n\nEl infractor tiene derecho a presentar alegaciones contra esta multa en un plazo de 15 días hábiles a partir de la fecha de la notificación. Las alegaciones deben presentarse por escrito y pueden ser entregadas en cualquiera de las oficinas de atención al ciudadano del Ayuntamiento o enviadas por correo postal a la dirección indicada en la notificación.\n\nNotificación:\n\nLa presente notificación se envía al titular de la vivienda registrada en el Ayuntamiento de Zaragoza. En caso de que los datos del titular no estén actualizados, se ruega que se actualicen a la mayor brevedad posible para evitar futuras incidencias."
      }
    ],
    "politicas_de_privacidad": [
      {
        "title": "Política de Privacidad - Ejemplo 1",
        "content": "POLÍTICA DE PRIVACIDAD DEL SITIO WEB I. POLÍTICA DE PRIVACIDAD Y PROTECCIÓN DE DATOS Respetando lo establecido en la legislación vigente, (en adelante, también Sitio Web) se compromete a adoptar las medidas técnicas y organizativas necesarias, según el nivel de seguridad adecuado al riesgo de los datos recogidos. Leyes que incorpora esta política de privacidad Esta política de privacidad está adaptada a la normativa española y europea vigente en materia de protección de datos personales en internet. En concreto, la misma respeta las siguientes normas: El Reglamento (UE) 2016/679 del Parlamento Europeo y del Consejo, de 27 de abril de 2016, relativo a la protección de las personas físicas en lo que respecta al tratamiento de datos personales y a la libre circulación de estos datos (RGPD). La Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales (LOPD-GDD). El Real Decreto 1720/2007, de 21 de diciembre, por el que se aprueba el Reglamento de desarrollo de la Ley Orgánica 15/1999, de 13 de diciembre, de Protección de Datos de Carácter Personal (RDLOPD). La Ley 34/2002, de 11 de julio, de Servicios de la Sociedad de la Información y de Comercio Electrónico (LSSI-CE). Identidad del responsable del tratamiento de los datos personales El responsable del tratamiento de los datos personales recogidos en es: , con NIF: (en adelante, Responsable del tratamiento). Sus datos de contacto son los siguientes: Dirección: Teléfono de contacto: Email de contacto: Registro de Datos de Carácter Personal En cumplimiento de lo establecido en el RGPD y la LOPD-GDD, le informamos que los datos personales recabados por , mediante los formularios extendidos en sus páginas quedarán incorporados y serán tratados en nuestro fichero con el fin de poder facilitar, agilizar y cumplir los compromisos establecidos entre y el Usuario o el mantenimiento de la relación que se establezca en los formularios que este rellene, o para atender una solicitud o consulta del mismo. Asimismo, de conformidad con lo previsto en el RGPD y la LOPD-GDD, salvo que sea de aplicación la excepción prevista en el artículo 30.5 del RGPD, se mantiene un registro de actividades de tratamiento que especifica, según sus finalidades, las actividades detratamiento llevadas a cabo y las demás circunstancias establecidas en el RGPD. Principios aplicables al tratamiento de los datos personales El tratamiento de los datos personales del Usuario se someterá a los siguientes principios recogidos en el artículo 5 del RGPD y en el artículo 4 y siguientes de la Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales: Principio de licitud, lealtad y transparencia: se requerirá en todo momento el consentimiento del Usuario previa información completamente transparente de los fines para los cuales se recogen los datos personales. Principio de limitación de la finalidad: los datos personales serán recogidos con fines determinados, explícitos y legítimos. Principio de minimización de datos: los datos personales recogidos serán únicamente los estrictamente necesarios en relación con los fines para los que son tratados. Principio de exactitud: los datos personales deben ser exactos y estar siempre actualizados. Principio de limitación del plazo de conservación: los datos personales solo serán mantenidos de forma que se permita la identificación del Usuario durante el tiempo necesario para los fines de su tratamiento. Principio de integridad y confidencialidad: los datos personales serán tratados de manera que se garantice su seguridad y confidencialidad. Principio de responsabilidad proactiva: el Responsable del tratamiento será responsable de asegurar que los principios anteriores se cumplen. Categorías de datos personales Las categorías de datos que se tratan en son únicamente datos identificativos. En ningún caso, se tratan categorías especiales de datos personales en el sentido del artículo 9 del RGPD. Base legal para el tratamiento de los datos personales La base legal para el tratamiento de los datos personales es el consentimiento. se compromete a recabar el consentimiento expreso y verificable del Usuario para el tratamiento de sus datos personales para uno o varios fines específicos. El Usuario tendrá derecho a retirar su consentimiento en cualquier momento. Será tan fácil retirar el consentimiento como darlo. Como regla general, la retirada del consentimiento no condicionará el uso del Sitio Web. En las ocasiones en las que el Usuario deba o pueda facilitar sus datos a través de formularios para realizar consultas, solicitar información o por motivos relacionados con el contenido del Sitio Web, se le informará en caso de que la cumplimentación de alguno de ellos sea obligatoria debido a que los mismos sean imprescindibles para el correcto desarrollo de la operación realizada. Fines del tratamiento a que se destinan los datos personales Los datos personales son recabados y gestionados por con la finalidad de poder facilitar, agilizar y cumplir los compromisos establecidos entre el Sitio Web y el Usuario o el mantenimiento de la relación que se establezca en los formularios que este último rellene o para atender una solicitud o consulta. Igualmente, los datos podrán ser utilizados con una finalidad comercial de personalización, operativa y estadística, y actividades propias del objeto social de , así como para la extracción, almacenamiento dedatos y estudios de marketing para adecuar el Contenido ofertado al Usuario, así como mejorar la calidad, funcionamiento y navegación por el Sitio Web. En el momento en que se obtengan los datos personales, se informará al Usuario acerca del fin o fines específicos del tratamiento a que se destinarán los datos personales; es decir, del uso o usos que se dará a la información recopilada. Períodos de retención de los datos personales Los datos personales solo serán retenidos durante el tiempo mínimo necesario para los fines de su tratamiento y, en todo caso, únicamente durante el siguiente plazo: , o hasta que el Usuario solicite su supresión. En el momento en que se obtengan los datos personales, se informará al Usuario acerca del plazo durante el cual se conservarán los datos personales o, cuando eso no sea posible, los criterios utilizados para determinar este plazo. Destinatarios de los datos personales Los datos personales del Usuario serán compartidos con los siguientes destinatarios o categorías de destinatarios: En caso de que el Responsable del tratamiento tenga la intención de transferir datos personales a un tercer país u organización internacional, en el momento en que se obtengan los datos personales, se informará al Usuario acerca del tercer país u organización internacional al cual se tiene la intención de transferir los datos, así como de la existencia o ausencia de una decisión de adecuación de la Comisión. Datos personales de menores de edad Respetando lo establecido en los artículos 8 del RGPD y 7 de la Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales, solo los mayores de 14 años podrán otorgar su consentimiento para el tratamiento de sus datos personales de forma lícita por . Si se trata de un menor de 14 años, será necesario el consentimiento de los padres o tutores para el tratamiento, y este solo se considerará lícito en la medida en la que los mismos lo hayan autorizado. Secreto y seguridad de los datos personales se compromete a adoptar las medidas técnicas y organizativas necesarias, según el nivel de seguridad adecuado al riesgo de los datos recogidos, de forma que se garantice la seguridad de los datos de carácter personal y se evite la destrucción, pérdida o alteración accidental o ilícita de datos personales transmitidos, conservados o tratados de otra forma, o la comunicación o acceso no autorizados a dichos datos. Sin embargo, debido a que no puede garantizar la inexpugnabilidad de internet ni la ausencia total de hackers u otros que accedan de modo fraudulento a los datos personales, el Responsable del tratamiento se compromete a comunicar al Usuario sin dilación indebida cuando ocurra una violación de la seguridad de los datos personales que sea probable que entrañe un alto riesgo para los derechos y libertades de las personas físicas. Siguiendo lo establecido en el artículo 4 del RGPD, se entiende por violación de la seguridad de los datos personales toda violación de la seguridad que ocasione la destrucción, pérdida o alteración accidental o ilícita de datos personales transmitidos, conservados o tratados de otra forma, o lacomunicación o acceso no autorizados a dichos datos. Los datos personales serán tratados como confidenciales por el Responsable del tratamiento, quien se compromete a informar de y a garantizar por medio de una obligación legal o contractual que dicha confidencialidad sea respetada por sus empleados, asociados, y toda persona a la cual le haga accesible la información. Derechos derivados del tratamiento de los datos personales El Usuario tiene sobre y podrá, por tanto, ejercer frente al Responsable del tratamiento los siguientes derechos reconocidos en el RGPD y la Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales: Derecho de acceso: Es el derecho del Usuario a obtener confirmación de si está tratando o no sus datos personales y, en caso afirmativo, obtener información sobre sus datos concretos de carácter personal y del tratamiento que haya realizado o realice, así como, entre otra, de la información disponible sobre el origen de dichos datos y los destinatarios de las comunicaciones realizadas o previstas de los mismos. Derecho de rectificación: Es el derecho del Usuario a que se modifiquen sus datos personales que resulten ser inexactos o, teniendo en cuenta los fines del tratamiento, incompletos. Derecho de supresión (\"el derecho al olvido\"): Es el derecho del Usuario, siempre que la legislación vigente no establezca lo contrario, a obtener la supresión de sus datos personales cuando estos ya no sean necesarios para los fines para los cuales fueron recogidos o tratados; el Usuario haya retirado su consentimiento al tratamiento y este no cuente con otra base legal; el Usuario se oponga al tratamiento y no exista otro motivo legítimo para continuar con el mismo; los datos personales hayan sido tratados ilícitamente; los datos personales deban suprimirse en cumplimiento de una obligación legal; o los datos personales hayan sido obtenidos producto de una oferta directa de servicios de la sociedad de la información a un menor de 14 años. Además de suprimir los datos, el Responsable del tratamiento, teniendo en cuenta la tecnología disponible y el coste de su aplicación, deberá adoptar medidas razonables para informar a los responsables que estén tratando los datos personales de la solicitud del interesado de supresión de cualquier enlace a esos datos personales. Derecho a la limitación del tratamiento: Es el derecho del Usuario a limitar el tratamiento de sus datos personales. El Usuario tiene derecho a obtener la limitación del tratamiento cuando impugne la exactitud de sus datos personales; el tratamiento sea ilícito; el Responsable del tratamiento ya no necesite los datos personales, pero el Usuario lo necesite para hacer reclamaciones; y cuando el Usuario se haya opuesto al tratamiento. Derecho a la portabilidad de los datos: En caso de que el tratamiento se efectúe por medios automatizados, el Usuario tendrá derecho a recibir del Responsable del tratamiento sus datos personales en un formato estructurado, de uso común y lectura mecánica, y a transmitirlos a otro responsable del tratamiento. Siempre que sea técnicamente posible, el Responsable del tratamiento transmitirá directamente los datos a ese otro responsable. Derecho de oposición: Es el derecho del Usuario a que no se lleve a cabo el tratamiento de sus datos de carácter personal o se cese el tratamiento de los mismos por parte de . Derecho a no ser objeto de una decisión basada únicamente en el tratamiento automatizado, incluida la elaboración de perfiles: Es el derecho del Usuario a no ser objeto de una decisión individualizada basada únicamente en el tratamiento automatizado de sus datos personales, incluida la elaboración de perfiles, existente salvo que la legislación vigente establezca lo contrario. Así pues, el Usuario podrá ejercitar sus derechos mediante comunicación escrita dirigida al Responsable del tratamiento con la referencia \"RGPD-\", especificando: Nombre, apellidos del Usuario y copia del DNI. En los casos en que se admita la representación, será también necesaria la identificación por el mismo medio de la persona que representa al Usuario, asícomo el documento acreditativo de la representación. La fotocopia del DNI podrá ser sustituida, por cualquier otro medio válido en derecho que acredite la identidad. Petición con los motivos específicos de la solicitud o información a la que se quiere acceder. Domicilio a efecto de notificaciones. Fecha y firma del solicitante. Todo documento que acredite la petición que formula. Esta solicitud y todo otro documento adjunto podrá enviarse a la siguiente dirección y/o correo electrónico: Dirección postal: Correo electrónico: Enlaces a sitios web de terceros El Sitio Web puede incluir hipervínculos o enlaces que permiten acceder a páginas web de terceros distintos de , y que por tanto no son operados por . Los titulares de dichos sitios web dispondrán de sus propias políticas de protección de datos, siendo ellos mismos, en cada caso, responsables de sus propios f icheros y de sus propias prácticas de privacidad. Reclamaciones ante la autoridad de control En caso de que el Usuario considere que existe un problema o infracción de la normativa vigente en la forma en la que se están tratando sus datos personales, tendrá derecho a la tutela judicial efectiva y a presentar una reclamación ante una autoridad de control, en particular, en el Estado en el que tenga su residencia habitual, lugar de trabajo o lugar de la supuesta infracción. En el caso de España, la autoridad de control es la Agencia Española de Protección de Datos (https://www.aepd.es/). II. ACEPTACIÓN Y CAMBIOS EN ESTA POLÍTICA DE PRIVACIDAD Es necesario que el Usuario haya leído y esté conforme con las condiciones sobre la protección de datos de carácter personal contenidas en esta Política de Privacidad, así como que acepte el tratamiento de sus datos personales para que el Responsable del tratamiento pueda proceder al mismo en la forma, durante los plazos y para las finalidades indicadas. El uso del Sitio Web implicará la aceptación de la Política de Privacidad del mismo. se reserva el derecho a modificar su Política de Privacidad, de acuerdo a su propio criterio, o motivado por un cambio legislativo, jurisprudencial o doctrinal de la Agencia Española de Protección de Datos. Los cambios o actualizaciones de esta Política de Privacidad no serán notificados de forma explícita al Usuario. Se recomienda al Usuario consultar esta página de forma periódica para estar al tanto de los últimos cambios o actualizaciones. Esta Política de Privacidad fue actualizada para adaptarse al Reglamento (UE) 2016/679 del Parlamento Europeo y del Consejo, de 27 de abril de 2016, relativo a la protección de las personas físicas en lo que respecta al tratamiento de datos personales y a la libre circulación de estos datos (RGPD) y a la Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales."
      },
      {
        "title": "Política de Privacidad - Ejemplo 2",
        "content": "POLÍTICA DE PRIVACIDAD\nEl presente Política de Privacidad establece los términos en que  usa y protege la información que es proporcionada por sus usuarios al momento de utilizar su sitio web. Esta compañía está comprometida con la seguridad de los datos de sus usuarios. Cuando le pedimos llenar los campos de información personal con la cual usted pueda ser identificado, lo hacemos asegurando que sólo se empleará de acuerdo con los términos de este documento. Sin embargo esta Política de Privacidad puede cambiar con el tiempo o ser actualizada por lo que le recomendamos y enfatizamos revisar continuamente esta página para asegurarse que está de acuerdo con dichos cambios.\n\nInformación que es recogida\n\nNuestro sitio web podrá recoger información personal por ejemplo: Nombre,  información de contacto como  su dirección de correo electrónica e información demográfica. Así mismo cuando sea necesario podrá ser requerida información específica para procesar algún pedido o realizar una entrega o facturación.\n\nUso de la información recogida\n\nNuestro sitio web emplea la información con el fin de proporcionar el mejor servicio posible, particularmente para mantener un registro de usuarios, de pedidos en caso que aplique, y mejorar nuestros productos y servicios.  Es posible que sean enviados correos electrónicos periódicamente a través de nuestro sitio con ofertas especiales, nuevos productos y otra información publicitaria que consideremos relevante para usted o que pueda brindarle algún beneficio, estos correos electrónicos serán enviados a la dirección que usted proporcione y podrán ser cancelados en cualquier momento.\n\n está altamente comprometido para cumplir con el compromiso de mantener su información segura. Usamos los sistemas más avanzados y los actualizamos constantemente para asegurarnos que no exista ningún acceso no autorizado.\n\nCookies\n\nUna cookie se refiere a un fichero que es enviado con la finalidad de solicitar permiso para almacenarse en su ordenador, al aceptar dicho fichero se crea y la cookie sirve entonces para tener información respecto al tráfico web, y también facilita las futuras visitas a una web recurrente. Otra función que tienen las cookies es que con ellas las web pueden reconocerte individualmente y por tanto brindarte el mejor servicio personalizado de su web.\n\nNuestro sitio web emplea las cookies para poder identificar las páginas que son visitadas y su frecuencia. Esta información es empleada únicamente para análisis estadístico y después la información se elimina de forma permanente. Usted puede eliminar las cookies en cualquier momento desde su ordenador. Sin embargo las cookies ayudan a proporcionar un mejor servicio de los sitios web, estás no dan acceso a información de su ordenador ni de usted, a menos de que usted así lo quiera y la proporcione directamente . Usted puede aceptar o negar el uso de cookies, sin embargo la mayoría de navegadores aceptan cookies automáticamente pues sirve para tener un mejor servicio web. También usted puede cambiar la configuración de su ordenador para declinar las cookies. Si se declinan es posible que no pueda utilizar algunos de nuestros servicios.\n\nEnlaces a Terceros\n\nEste sitio web pudiera contener en laces a otros sitios que pudieran ser de su interés. Una vez que usted de clic en estos enlaces y abandone nuestra página, ya no tenemos control sobre al sitio al que es redirigido y por lo tanto no somos responsables de los términos o privacidad ni de la protección de sus datos en esos otros sitios terceros. Dichos sitios están sujetos a sus propias políticas de privacidad por lo cual es recomendable que los consulte para confirmar que usted está de acuerdo con estas.\n\nControl de su información personal\n\nEn cualquier momento usted puede restringir la recopilación o el uso de la información personal que es proporcionada a nuestro sitio web.  Cada vez que se le solicite rellenar un formulario, como el de alta de usuario, puede marcar o desmarcar la opción de recibir información por correo electrónico.  En caso de que haya marcado la opción de recibir nuestro boletín o publicidad usted puede cancelarla en cualquier momento.\n\nEsta compañía no venderá, cederá ni distribuirá la información personal que es recopilada sin su consentimiento, salvo que sea requerido por un juez con un orden judicial.\n\n Se reserva el derecho de cambiar los términos de la presente Política de Privacidad en cualquier momento."
      },
      {
        "title": "Política de Privacidad - Ejemplo 3",
        "content": "POLITICA DE PRIVACIDAD\n\n1. INFORMACIÓN AL USUARIO\n\nTU EMPRESA, S.L., como Responsable del Tratamiento, le informa que, según lo dispuesto en el Reglamento (UE) 2016/679, de 27 de abril, (RGPD) y en la L.O. 3/2018, de 5 de diciembre, de protección de datos y garantía de los derechos digitales (LOPDGDD), trataremos su datos tal y como reflejamos en la presente Política de Privacidad.\n\nEn esta Política de Privacidad describimos cómo recogemos sus datos personales y por qué los recogemos, qué hacemos con ellos, con quién los compartimos, cómo los protegemos y sus opciones en cuanto al tratamiento de sus datos personales.\n\nEsta Política se aplica al tratamiento de sus datos personales recogidos por la empresa para la prestación de sus servicios. Si acepta las medidas de esta Política, acepta que tratemos sus datos personales como se define en esta Política.\n\n2. CONTACTO\n\nDenominación social: TU EMPRESA, S.L.\nNombre comercial: TU EMPRESA\nCIF: NUMERO CIF\nDomicilio: DIRECCION DE TU NEGOCIO\nE-mail: tuemail@tudominio. com\n\n3. PRINCIPIOS CLAVE\n\nSiempre hemos estado comprometidos con prestar nuestros servicios con el más alto grado de calidad, lo que incluye tratar sus datos con seguridad y transparencia. Nuestros principios son:\n\nLegalidad: Solo recopilaremos sus Datos personales para fines específicos, explícitos y legítimos.\nMinimización de datos: Limitamos la recogida de datos de carácter personal a lo que es estrictamente relevante y necesario para los fines para los que se han recopilado.\nLimitación de la Finalidad: Solo recogeremos sus datos personales para los fines declarados y solo según sus deseos.\nPrecisión: Mantendremos sus datos personales exactos y actualizados.\nSeguridad de los Datos: Aplicamos las medidas técnicas y organizativas adecuadas y proporcionales a los riesgos para garantizar que sus datos no sufran daños, tales como divulgación o acceso no autorizado, la destrucción accidental o ilícita o su pérdida accidental o alteración y cualquier otra forma de tratamiento ilícito.\nAcceso y Rectificación: Disponemos de medios para que acceda o rectifique sus datos cuando lo considere oportuno.\nConservación: Conservamos sus datos personales de manera legal y apropiada y solo mientras es necesario para los fines para los que se han recopilado.\nLas transferencias internacionales: cuando se dé el caso de que sus datos vayan a ser transferidos fuera de la UE/EEE se protegerán adecuadamente.\nTerceros: El acceso y transferencia de datos personales a terceros se llevan a cabo de acuerdo con las leyes y reglamentos aplicables y con las garantías contractuales adecuadas.\nMarketing Directo y cookies: Cumplimos con la legislación aplicable en materia de publicidad y cookies.\n\n4. RECOGIDA Y TRATAMIENTO DE SUS DATOS PERSONALES\nLas tipos de datos que se pueden solicitar y tratar son:\n\nDatos de carácter identificativo.\nTambién recogemos de forma automática datos sobre su visita a nuestro sitio web  según se describe en la política de cookies.\n\nSiempre que solicitemos sus Datos personales, le informaremos con claridad de qué datos personales recogemos y con qué fin. En general, recogemos y tratamos sus datos personales con el propósito de:\n\nProporcionar información, servicios, productos, información relevante y novedades en el sector.\nEnvío de comunicaciones.\n\n5. LEGITIMIDAD\n\nDe acuerdo con la normativa de protección de datos aplicable, sus datos personales podrán tratarse siempre que:\n\nNos ha dado su consentimiento a los efectos del tratamiento. Por supuesto podrá retirar su consentimiento en cualquier momento.\nPor requerimiento legal.\nPor exisitr un interés legítimo que no se vea menoscabado por sus derechos de privacidad, como por ejemplo el envío de información comercial bien por suscripción a nuestra newsletter o por su condición de cliente.\nPor se necesaria para la prestación de alguno de nuestros servicios mediante relación contractual entre usted y nosotros.\n\n6. COMUNICACIÓN DE DATOS PERSONALES\n\nLos datos pueden ser comunicados a empresas relacionadas con TU EMPRESA, S.L. para la prestación de los diversos servicios en calidad de Encargados del Tratamiento. La empresa no realizará ninguna cesión, salvo por obligación legal.\n\n7. SUS DERECHOS\n\nEn relación con la recogida y tratamiento de sus datos personales, puede ponerse en contacto con nosotros en cualquier momento para:\n\nAcceder a sus datos personales y a cualquier otra información indicada en el Artículo 15.1 del RGPD.\nRectificar sus datos personales que sean inexactos o estén incompletos de acuerdo con el Artículo 16 del RGPD.\nSuprimir sus datos personales de acuerdo con el Artículo 17 del RGPD.\nLimitar el tratamiento de sus datos personales de acuerdo con el Artículo 18 del RGPD.\nSolicitar la portabilidad de sus datos de acuerdo con el Artículo 20 del RGPD.\nOponerse al tratamiento de sus datos personales de acuerdo con el artículo 21 del RGPD.\nSi ha otorgado su consentimiento para alguna finalidad concreta, tiene derecho a retirar el consentimiento otorgado en cualquier momento, sin que ello afecte a la licitud del tratamiento basado en el consentimiento previo a su retirada rrhh.\n\nPuede ejercer estos derechos enviando comunicación, motivada y acreditada, a tuemail@tudominio .com\n\nTambién tiene derecho a presentar una reclamación ante la Autoridad de control competente (www.aepd.es) si considera que el tratamiento no se ajusta a la normativa vigente.\n\n8. INFORMACIÓN LEGAL\nLos requisitos de esta Política complementan, y no reemplazan, cualquier otro requisito existente bajo la ley de protección de datos aplicable, que será la que prevalezca en cualquier caso.\n\nEsta Política está sujeta a revisiones periódicas y la empresa puede modificarla en cualquier momento. Cuando esto ocurra, le avisaremos de cualquier cambio y le pediremos que vuelva a leer la versión más reciente de nuestra Política y que confirme su aceptación."
      }
    ],
    "contratos": [
      {
        "title": "Contrato - Ejemplo 1",
        "content": "Contrato de compraventa de un vehículo usado entre particulares\nEn ........................... a.......... de .................... de 20...... HORA: ………………………..\nVendedor:\nD. ……………………………….…….……………………., con N.I.F. nº …………….……….………, y domicilio en ……………………….…., calle de ………………….……………..\nComprador:\nD. ………………………….…………….……….…………, con N.I.F. nº ……………….………………, y domicilio en........................................…., calle de…………….……….…….Vehículo:\nMarca: ...........................\nMatrícula: ...........................\nNº de Bastidor: …………………………………..\nKilómetros:………………….Reunidos vendedor y comprador en la fecha del encabezamiento, manifiestan haber acordado formalizar en este documento CONTRATO DE COMPRAVENTA del vehículo automóvil que se especifica, en las siguientes\nCONICIONES\n1ª) El vendedor vende al comprador el vehículo de su propiedad anteriormente especificado por la cantidad de …………… euros, sin incluir los impuestos correspondientes, que serán a cargo del comprador.\n2ª) El vendedor declara que no pesa sobre el vehículo ninguna carga o gravamen ni impuesto, deuda o sanción pendientes de abono en la fecha de la firma de este contrato, comprometiéndose en caso contrario a regularizar tal situación a su exclusivo cargo.\n3ª) El vendedor se compromete a facilitar los distintos documentos relativos al vehículo, así como a firmar cuantos documentos aparte de éste sean necesarios para que el vehículo quede correctamente inscrito a nombre del comprador en los correspondientes organismos públicos, siendo todos los gastos a cargo del comprador.\n4ª) Una vez realizada la correspondiente transferencia en Tráfico, el vendedor entregará materialmente al comprador la posesión del vehículo, haciéndose el comprador cargo de cuantas responsabilidades puedan contraerse por la propiedad del vehículo y su tenencia y uso a partir de dicho momento de la entrega.\n5ª) El vehículo dispone de seguro en vigor hasta fecha de………………… y se encuentra al corriente respecto a las obligaciones derivadas de la ITV (Inspección Técnica de Vehículos).\n6ª) El comprador declara conocer el estado actual del vehículo, por lo que exime al vendedor de garantía por vicios o defectos que surjan con posterioridad a la entrega, salvo aquellos ocultos que tengan su origen en dolo o mala fe del vendedor.\n7ª) Para cualquier litigio que surja entre las partes de la interpretación o cumplimiento del presente contrato, éstas, con expresa renuncia al fuero que pudiera corresponderles, se someterán a los Juzgados y Tribunales de............................\nY para que así conste, firman el presente contrato de compraventa, por triplicado, en la fecha y lugar arriba indicados.\nFirma del vendedor Firma del comprador"
      },
      {
        "title": "Contrato - Ejemplo 2",
        "content": "CONTRATO DE COMPRAVENTA DE VIVIENDA 1\nEn ......, a ...... de ...... de ...... (Lugar y fecha)\n\nREUNIDOS\n\nDE UNA PARTE:\n\n.............\n\nY DE OTRA PARTE:\n\n.............\n\nINTERVIENEN: Todos en su propio nombre y derecho. Ambas partes se reconocen mutuamente la capacidad legal necesaria para el otorgamiento y firma del presente contrato de compraventa, para lo cual,\n\nEXPONEN\n\nI\n\nQue ...... (Datos del propietario que vende) es propietario de la siguiente finca:\n\n...... (Descripción de la vivienda)\n\nTÍTULO DE PROPIEDAD: La finca descrita fue adquirida por los vendedores a\n\n......\n\nINSCRIPCIÓN: La finca se encuentra inscrita en el Registro de la Propiedad de\n\n.......\n\nCARGAS:\n\n...... (Detallar)\n\nII\n\nQue estando de acuerdo los abajo firmantes en que D/ña. ...... (Datos del vendedor) venda a D/ña. ...... la finca citada y descrita en el Expositivo anterior, y siendo voluntad de éste adquirirla, llevan a efecto de común acuerdo la presente compraventa de conformidad con las siguientes,\n\nCLÁUSULAS\n\nPRIMERA.- El vendedor ....... vende a D/ña. ......., quien compra la finca descrita en el Expositivo primero.\n\nSEGUNDA.- El precio de esta compraventa se fija en ....... euros, importe que se abona de la siguiente forma:\n\n...... euros como señal de la compraventa 2 , que se entregan en el acto de la firma del presente contrato, respecto a las cuales el vendedor declara haberlas percibido y otorga carta de pago a favor de D/ña. ........\nEl resto, ....... euros, se entregarán por la parte compradora en efectivo en el momento del otorgamiento de la correspondiente escritura pública de esta compraventa. Convienen, por tanto, las partes en elevar a escritura pública el presente contrato en un plazo no superior a tres meses a partir de la fecha de la firma del presente contrato, ante el Notario que de común parecer acuerden; bastará el requerimiento de cualquiera de las partes a la otra para ello.\nTERCERA.- La entrega de las llaves del piso a la parte compradora se llevará a efecto en el momento de la elevación a público del presente contrato.\n\nCUARTA.- La parte compradora se compromete a hacerse cargo de la totalidad de los gastos que la titularidad del inmueble lleva consigo, especialmente los gastos de comunidad, a partir de la entrega de llaves a la parte compradora.\n\nQUINTA.- Respecto a los demás gastos e impuestos que se originen o devenguen como consecuencia de la presente compraventa, éstos serán los que por ley correspondan a cada una de las partes.\n\nSEXTA.- El vendedor o vendedores responderán ante el comprador por evicción y saneamiento con arreglo a las prescripciones legales que regulen estas materias.\n\nSÉPTIMA.- Se hace constar expresamente que existe sobre la vivienda seguro decenal de daños materiales (o seguro de caución) contratado por el promotor con la compañía .......\n\nY en prueba de conformidad, firman ambas partes el presente contrato, por duplicado en todas sus hojas, en el lugar y fecha indicados en el encabezamiento.\n\nLa parte vendedora,\n\n (Firma y nombre)\n\nLa parte compradora,\n\n(Firma y nombre)\n\nNotas\n\n1 Cada una de las páginas del documento deben ser firmadas por el comprador y vendedor (independientemente de las firmas de la última página).\n\n2 Señal: Como señal, entendemos la cantidad entregada por el comprador al vendedor a cuenta del precio de la vivienda. Aunque habitualmente la manera de formalizarla sea mediante un \"recibí\" (escrito en un papel), recomendamos que se realice dentro de un contrato completo. Aquello que todo el mundo conoce como \"señal\" no siempre lo es. Se pueden distinguir tres tipos de pagos a cuenta.\n2.1- Señal propiamente dicha. Es la cantidad entregada anticipadamente por el comprador como parte del precio. Equivale a una confirmación del contrato, por lo que si una de las partes quiere echarse atrás, la otra puede exigirle el cumplimiento, incluso pedir daños y perjuicios. Esta opción interesa al vendedor en el caso de que esté seguro de querer vender en las condiciones pactadas en el contrato.\n\n2.2- Arras penitenciales. Es un pacto expreso (de otro modo sería una señal propiamente dicha) por el que cualquiera de las partes puede apartarse del cumplimiento del contrato: el vendedor, si devuelve duplicada la cantidad entregada, y el comprador, si renuncia a la cantidad entregada a cuenta. Debe mencionarse en la cláusula para mayor seguridad la referencia al artículo 1.454 del Código Civil que es la que describe la figura de las arras penitenciales. Un modelo de cláusula sustitutoria sería la siguiente:\n\"En este acto, y por la compra del inmueble descrito, el comprador entrega al vendedor la cantidad de 3.000 euros en concepto de arras penitenciales, en virtud de las cuales se reconocen la posibilidad resolutoria del pacto de compraventa que suscriben en la forma y efectos previstos en el art. 1.454 del Código Civil\".\n\n2.3- Arras penales. Es una variedad de la señal propiamente dicha. Se trata de un pacto que estipula cuál será la indemnización a pagar por las partes en caso de incumplimiento. Como la señal, interesa al vendedor que esté seguro de haber realizado una buena venta. Una cláusula tipo sería la siguiente:\n\"En este acto, y por la compra del inmueble descrito, el comprador entrega al vendedor la cantidad de 3.000 euros en concepto de pago parcial anticipado a cuenta del precio pactado, cantidad por la que el vendedor otorga en este documento carta de pago. Ambas partes establecen de mutuo acuerdo para caso de incumplimiento:\nDel vendedor: la devolución de la cantidad de 3.000 euros más el 50% de indemnización, en concepto de arras penales.\nDel comprador: su pérdida.\nY sin perjuicio en ambos casos del derecho que asiste a la otra parte de exigir el cumplimiento del contrato\".\nLo consignado en el texto del modelo es una opción, la primera, es decir, la señal propiamente dicha. Si desea o le conviene alguna de las otras dos, adapte en la medida necesaria dicha cláusula segunda.\n\n3 En la actualidad los diferentes gastos se estructuran de la forma siguiente: Gastos del vendedor.\nSi ha existido un agente mediador, (API, etc.), su comisión corresponde al vendedor, salvo que se pacte otra cosa. Respecto a los gastos de notario, si bien lo habitual es que por pacto toda la factura la pague el comprador, legalmente si nada se indica los gastos del otorgamiento de escrituras serán de cuenta del vendedor (matriz), y los de primera copia (copia autorizada) y las demás posteriores a la venta serán de cuenta del comprador.\nCertificado de la comunidad de propietarios sobre el estado de deudas. Es obligatorio, salvo que el comprador renuncie a él.\nCancelación de hipoteca pendiente (si la tiene, y no se subroga el comprador en la hipoteca).\nImpuestos.\nEl Impuesto de Plusvalía, o Impuesto sobre el Incremento de Valor de los Terrenos de Naturaleza Urbana, corresponde pagarlo al vendedor (salvo en Navarra y cuando el vendedor resida fuera de España), si bien es posible trasladar su coste al comprador, incluyéndolo en el precio de venta o mediante una cláusula en el contrato que lo indique. Sin embargo, los pactos entre comprador y vendedor no vinculan al Ayuntamiento: si el comprador no paga el Ayuntamiento reclamará el pago al vendedor.\nEl Impuesto sobre Bienes Inmuebles (IBI), de cara al Ayuntamiento corresponde por ley pagarlo a quien sea propietario del inmueble el día uno de enero del año que corresponda. Sin embargo, el vendedor tiene derecho a que sea el comprador quien cargue con la parte proporcional del Impuesto desde el día de la transmisión del inmueble, salvo que pacten otra cosa. El notario tiene la obligación de comunicar la venta al Catastro.\nVea además la parte correspondiente a Impuestos en los gastos del comprador.\nGastos del comprador.\nGastos de Notaría. Vea lo dicho en los gastos del vendedor.\nGastos de Registro de la Propiedad.\nImpuestos.\nSi la vivienda es nueva, procede el Impuesto sobre el Valor Añadido, IVA, que asciende al 10 %, salvo para el caso de viviendas de protección pública en régimen especial, cuyo tipo es el 4%. Además, ha de abonarse el Impuesto de Actos Jurídicos Documentados, AJD (depende de la Comunidad Autónoma),\nEn el caso de vivienda usada, procede el Impuesto de Transmisiones Patrimoniales, ITP (según Comunidades, normalmente entre un 6-11 %).\n\n4 Saneamiento y evicción.\nPara viviendas nuevas, los plazos de garantía son de 10,3 y 1 año a partir de la suscripción del acta de recepción de la obra dependiendo del tipo de defecto. Los promotores de vivienda deben contratar un seguro que garantice el cumplimiento de las obligaciones del plazo de garantía decenal. Si el defecto se debe a un incumplimiento del contrato de venta, el plazo para reclamar es de 5 años. En vivienda usada, en principio solo se puede exigir responsabilidad por vicios ocultos al vendedor durante 6 meses desde la fecha de la venta. Vea nota 5.\n\n5 El seguro decenal es obligatorio para edificaciones con destino principal de vivienda, cuya licencia de edificación (obras) se haya solicitado a partir del 6 de mayo del año 2000, fecha de entrada en vigor de la Ley de Ordenación de la Edificación. Por ello, sólo se incluirá cuando proceda."
      },
      {
        "title": "Contrato - Ejemplo 3",
        "content": "CONTRATO DE ARRENDAMIENTO DE VIVIENDA\nEn …………,  a  …….. de  …………. de 20…\n\nREUNIDOS\nDe una parte, y como ARRENDADOR, persona física, D/Dña. ……………….……………., mayor de edad, de estado civil …..……., domiciliado/a en ……………………………….…, y con NIF nº …………….... A efectos de comunicaciones corrientes, como complemento al domicilio indicado, señala como correo electrónico …………...@..............., y el siguiente número de teléfono:………………..\n\nDe otra parte, y como ARRENDATARIO, D/Dña. ………..…………………, mayor de edad, de estado civil …………., domiciliado en ……………………………..…, y con NIF nº ……………... A efectos de comunicaciones corrientes, además de la vivienda que es objeto de este contrato, señala como correo electrónico…………....@................, y el siguiente número de teléfono:………..……..\n\nINTERVIENEN\nTodas las partes en su propio nombre y representación.\n\nAmbas partes se reconocen la capacidad legalmente necesaria para el otorgamiento del presente contrato de arrendamiento, y a tal efecto,\n\nEXPONEN\n1º. Que  D/Dña. …………………………. es propietario de la vivienda sita en …(dirección completa)…..……………..………………., con una superficie aproximada de …………… m2 útiles, inscrita en el Registro de la Propiedad de ……………., y con referencia catastral número: ………………….. La vivienda posee el preceptivo certificado de eficiencia energética y contiene el mobiliario que se indica en el inventario adjunto. Como anexos que se alquilan junto con la vivienda citada figuran: …………………………….…. (Identificar, si es el caso, una plaza de garaje, trastero, etc.). La vivienda se encuentra en la actualidad libre de ocupantes.\n\n2º. Que teniendo interés D/Dña. …………………………., en habitar la vivienda en calidad de arrendatario y el arrendador en arrendar la vivienda, ambas partes acuerdan la celebración del presente CONTRATO DE ARRENDAMIENTO DE VIVIENDA, y ello conforme a las siguientes:\n\nCLÁUSULAS\nPRIMERA.- El arrendamiento se establece por un plazo de …………. (el plazo que se acuerde), a contar desde el día ............... (fecha del contrato o de la ocupación de la vivienda si es posterior a la primera). De acuerdo con la legislación vigente a la firma de este contrato, el arrendatario tendrá la facultad de su renovación anual hasta que el arrendamiento alcance la duración de 5 años (1); no habrá lugar a dicha renovación si el arrendatario notificase por escrito la renuncia con al menos 30 días de antelación a la finalización del plazo pactado o cualquiera de sus prórrogas anuales.\n\nLlegada la fecha de vencimiento del contrato o de cualquiera de sus prórrogas, una vez transcurridos como mínimo cinco años de duración del contrato, si ninguna de las partes hubiese notificado a la otra, al menos con cuatro meses de antelación a aquella fecha en el caso del arrendador y al menos con dos meses de antelación en el caso del arrendatario, su voluntad de no renovarlo, el contrato se prorrogará obligatoriamente por plazos anuales hasta un máximo de tres años más, salvo que el arrendatario manifieste al arrendador con un mes de antelación a la fecha de terminación de cualquiera de las anualidades, su voluntad de no renovar el contrato.\n\nEn virtud de las disposiciones de la Ley 12/2023, de 24 de mayo, por el Derecho a la Vivienda, y del artículo 10.3 de la Ley de Arrendamientos Urbanos vigente, se reconoce al arrendatario el derecho legal que le pueda asistir a prorrogar a su demanda el presente contrato en el caso de que la zona donde se ubique la vivienda alquilada esté afectada por una declaración vigente de zona de mercado residencial tensionado en los términos de dicha Ley de Vivienda.\n\nSEGUNDA. - Expresamente se pacta un periodo mínimo de validez del contrato de seis meses. El incumplimiento de dicho plazo por el arrendatario dará lugar a una indemnización dineraria equivalente al periodo de renta que falte por cumplir para llegar al periodo mínimo pactado. Pasado el periodo mínimo, el arrendatario podrá desistir en todo momento del contrato siempre que realice un preaviso con treinta días de antelación. Realizado el preaviso con la antelación pactada, queda convenido el pago de una indemnización por el arrendatario, en concepto de perjuicios para el propietario, equivalente a un mes de renta por cada año de contrato que reste por cumplir. Los periodos de tiempo inferiores al año darán lugar a la parte proporcional de la indemnización. En el caso de que no se hubiese realizado dicho preaviso en el tiempo convenido, la indemnización a pagar al arrendador se verá incrementada en la cantidad equivalente a un mes de renta.\n\nEl arrendador hace constar expresamente que no procederá la prórroga obligatoria del contrato en el caso de que, trascurrido el primer año de contrato, se ponga de manifiesto la necesidad para el arrendador de ocupar la vivienda arrendada antes del transcurso de cinco años para destinarla a vivienda permanente para sí o sus familiares de primer grado de consanguinidad o por adopción, o para su cónyuge en los supuestos de sentencia firme de separación, divorcio o nulidad matrimonial. El arrendador deberá comunicar este extremo con al menos dos meses de antelación a la fecha en que la vivienda se vaya a necesitar. En ese caso, el arrendatario estará obligado a entregar la vivienda en dicho plazo sin que ello genere derecho a indemnización alguna por parte del arrendador.\n\nTERCERA. -  Como precio del arrendamiento, se fija como renta anual la cantidad de …….. euros, pagadera mensualmente por meses anticipados, dentro de los siete primeros días de cada mes. El precio del arrendamiento incluye los gastos generales del inmueble, servicios, tributos, cargas y demás responsabilidades no susceptibles de individualización (2), pero no así los gastos por servicios con que cuenta la vivienda arrendada y que se individualicen (3) mediante aparatos contadores, que serán de cuenta exclusiva del arrendatario. Queda expresamente convenido que la tasa municipal anual de basuras sea por cuenta del arrendatario.\n\nLa renta se actualizará anualmente, en la fecha en que se cumpla cada año de vigencia de contrato, conforme (4) a la variación del Índice General Nacional de Precios al Consumo (el IPC general) en el período de los 12 meses anteriores a la fecha de celebración del contrato (5). Para la primera actualización se toma como mes de referencia el último índice publicado a la fecha de celebración del contrato, que es el del mes de ………….. de ……., (índice ………). Las sucesivas actualizaciones a partir de la primera se realizarán sobre la renta anteriormente actualizada. La renta actualizada será exigible al arrendatario a partir del mes siguiente a aquel en que la parte interesada lo notifique a la otra parte por escrito, expresando el porcentaje de alteración aplicado.\n\nCUARTA. - El pago se realizará mediante ingreso en metálico o transferencia bancaria a favor del Arrendador en la cuenta que a nombre de este último y con el nº ......................... existe en el Banco (Caja) .........., sucursal nº ..... . Dicho pago se acreditará de manera suficiente mediante el oportuno resguardo del ingreso o transferencia realizados, sin necesidad de expedir recibo alguno por parte del arrendador (6).\n\nQUINTA. - Consecuencias del impago de la renta o de cantidades asimiladas a la renta\n\nLa falta de pago y el retraso en el pago, tanto de la renta como del resto de las cantidades debidas, se consideran causas expresas de resolución del presente contrato. El arrendatario se obliga en tal caso a restituir inmediatamente el inmueble al arrendador. Expresamente se acuerda que la renta adeudada y no satisfecha generará un interés de demora anual equivalente al interés legal del dinero más dos puntos. Todos los gastos derivados de una reclamación judicial o extrajudicial por incumplimiento serán por cuenta de la parte deudora.\n\nEl Arrendador comunica al Arrendatario que en caso de impago de la renta o de las cantidades asimiladas a la renta, sus datos personales podrán ser cedidos a ficheros para la evaluación de la solvencia patrimonial y crédito y a ficheros comunes para la gestión y prevención del fraude. Dicha incorporación a los citados ficheros se realizará respetando la normativa sobre protección de datos personales. Antes de la incorporación, el Arrendador enviará un requerimiento de pago al inquilino, recordando esta posible inclusión de los datos en el fichero, y tras la incorporación de los datos de impagos del Arrendatario, este recibirá una comunicación del titular del fichero, en el plazo máximo de un mes desde la inscripción de la deuda en el fichero, informándole de la incorporación de sus datos personales en el mismo y de sus derechos de acceso, rectificación, cancelación y oposición.\n\nSEXTA. - El Arrendatario declara recibir la vivienda en perfecto estado de uso y se obliga a entregarla al término del contrato en el mismo estado de conservación, salvo el desgaste por un uso diligente ordinario. Todas las reparaciones y obras que, como consecuencia del uso ordinario hayan de realizarse en la vivienda, sus elementos, instalaciones y electrodomésticos, serán por cuenta del Arrendatario salvo las que legalmente le correspondan al Arrendador. Este está obligado a realizar las reparaciones que sean necesarias para conservar la vivienda en condiciones de habitabilidad, salvo que aquellas sean consecuencia de un deterioro imputable al arrendatario, sus dependientes o visitantes, por acción, omisión o negligencia.\n\nEl Arrendatario deberá poner en conocimiento del Arrendador, en el plazo más breve posible, la necesidad de las reparaciones que sean precisas para conservar la vivienda en las condiciones de habitabilidad, permitiendo a estos efectos el acceso a la vivienda del Arrendador o de las personas encargadas de la reparación.\n\nSÉPTIMA. - Sin perjuicio de lo previsto en la Ley de Arrendamientos Urbanos para arrendatarios con discapacidad o mayores de 70 años, el Arrendatario no podrá realizar en la vivienda obras que modifiquen su configuración o la de cualquiera de sus accesorios, sin el permiso de la parte arrendadora expresado por escrito. La contravención de esta cláusula será causa de resolución del presente contrato y el arrendador podrá exigir la reposición de las cosas al estado anterior. Las obras autorizadas se realizarán a expensas del arrendatario y, al término del contrato, quedarán a beneficio de la propiedad sin generar derecho a indemnización alguna.\n\nDe acuerdo con el artículo 19.4 de la Ley de Arrendamientos Urbanos, en cualquier momento desde el inicio de la vigencia del presente contrato de arrendamiento y previo acuerdo entre arrendador y arrendatario, podrán realizarse obras de mejora en la vivienda arrendada e incrementarse la renta del contrato, sin que ello implique la interrupción del periodo de prórroga obligatoria establecido en el artículo 9 o de prórroga tácita a que se refiere el artículo 10 de la LAU.\n\nLa realización por el arrendador de obras de mejora transcurridos cinco años de duración del contrato le dará derecho a elevar la renta anual en la cuantía que resulte de aplicar al capital invertido en la mejora (descontadas las subvenciones públicas obtenidas para la realización de la obra) el tipo de interés legal del dinero en el momento de la terminación de las obras incrementado en tres puntos, sin que pueda exceder el aumento del veinte por ciento de la renta vigente en aquel momento.\n\nOCTAVA. -El arrendatario se compromete y obliga expresamente a:\n\nDestinar el inmueble alquilado a vivienda permanente, excluyendo otros usos como, por ejemplo, despacho, oficina, alquiler temporal o vacacional, desarrollo de actividades profesionales o empresariales, etc.\nRespetar las normas de convivencia de la comunidad de propietarios.\nque en la vivienda no se realicen actividades de tipo industrial o actividades molestas, insalubres, nocivas o peligrosas;\nno ceder ni subarrendar la vivienda sin el permiso expreso por escrito del arrendador;\nSe prohíbe la tenencia de animales peligrosos en el piso, salvo previo permiso expreso por escrito del arrendador.\nLa contravención de cualquiera de estas obligaciones, constatada por el arrendador, constituirá causa de resolución del contrato, al igual que el resto de las causas de resolución previstas en la ley o en el presente contrato.\n\nNOVENA. - En el momento de la firma de este contrato, el Arrendatario hace entrega del importe de una mensualidad de renta, ........ euros, en concepto de fianza legal. Además, como garantía complementaria, el arrendatario hace entrega igualmente a la firma de este contrato del importe de otras dos mensualidades de renta, ……. euros. Tanto la fianza legal como la garantía complementaria quedan establecidas en garantía de las obligaciones legales y contractuales del arrendatario. El arrendador se compromete a realizar el depósito de la fianza legal según la normativa autonómica aplicable, y entregará copia del resguardo de dicho depósito al arrendatario, a petición de este.\n\nDÉCIMA. -  El arrendatario reconoce la veracidad y certeza del inventario que del mobiliario de la vivienda se relaciona al final del contrato, y se compromete a devolver la posesión de la vivienda y todo el mobiliario cuando corresponda, en su integridad y en perfecto estado de conservación. Se conviene la afección de la fianza que se entrega en el acto de la firma como garantía del cumplimiento de esta obligación, y además se compromete el arrendatario a entregar al propietario el importe de los daños causados que sobrepasen esta cuantía.\n\nEn particular, el arrendatario se compromete a facilitar, al término del contrato, los trámites para modificar la titularidad de los suministros, respondiendo de los daños que pueda generar al arrendador, por ejemplo, por dar de baja alguno de los contratos sin el acuerdo escrito del arrendador.\n\nUNDÉCIMA. - En caso de venta de la vivienda arrendada, las partes acuerdan la renuncia por parte del arrendatario al derecho de adquisición preferente, de acuerdo con la previsión establecida en el artículo 25.8 de la Ley de Arrendamientos Urbanos. El arrendador se compromete a comunicar al arrendatario su intención de vender la vivienda con una antelación mínima de treinta días a la fecha de formalización del contrato de compraventa. Realizada la venta, el arrendatario tendrá los derechos que le reconoce la LAU para estos supuestos, quedando el adquirente subrogado en los derechos y obligaciones del arrendador durante los cinco primeros años de vigencia del contrato.\n\nDÉCIMO SEGUNDA. - A los efectos legales oportunos, se hace constar que a la firma del presente contrato el arrendador ha entregado al arrendatario una copia del certificado de eficiencia energética correspondiente al inmueble arrendado.\n\nEn prueba de total conformidad, las partes firman el presente contrato, que se extiende por duplicado, en el lugar y fecha arriba indicados.\n\nEL (LOS) ARRENDADOR(ES),                             EL (LOS) ARRENDATARIO(S),\n\nFirma ................................                                        Firma ................................    \n\nNota 1. La referencia a la duración mínima de 5 años se refiere a contratos de arrendamiento de vivienda en los que el arrendador sea persona física. Si el arrendador es una persona jurídica, todas las referencias a la duración mínima deben recoger el plazo de 7 años en lugar del de 5.\n\nNota 2.\n\nRespecto de la cuantía de la renta ten en cuenta que si la vivienda se ubica en una zona de mercado residencial tensionado dentro del periodo de vigencia de la declaración de la referida zona en los términos dispuestos en la Ley 12/2023, de 24 de mayo, por el derecho a la vivienda, la renta pactada al inicio del nuevo contrato no podrá exceder de la última renta de contrato de arrendamiento de vivienda habitual que hubiese estado vigente en los últimos cinco años en la misma vivienda, una vez aplicada la cláusula de actualización anual de la renta del contrato anterior, sin que se puedan fijar nuevas condiciones que establezcan la repercusión al arrendatario de cuotas o gastos que no estuviesen recogidas en el contrato anterior.\n\nSi la casa no estaba alquilada en los últimos cinco años, la renta estará limitada por el índice de precios de referencia que sea aplicable según la resolución oficial que declare la zona como de mercado residencial tensionado.\n\nCuando la vivienda se encuentre en una zona declarada de mercado tensionado, el contrato lo indicará y precisará la cuantía de la última renta del contrato de arrendamiento de vivienda habitual que hubiese estado vigente en los últimos cinco años en la misma vivienda, así como del valor que le pueda corresponder atendiendo al índice de referencia de precios de alquiler de viviendas que resulte de aplicación.\n\nNota 3. En el caso de que estos gastos quieran cargarse al arrendatario, tenga en cuenta lo siguiente. Hay que señalar el importe exacto pagado el último año y pactar que se incrementarán anualmente conforme a la variación que experimenten, pero que nunca podrá alcanzar el doble del IPC General; en este caso, la cláusula a insertar en sustitución de esta sería la siguiente:\n\nEl arrendatario se obliga a abonar el importe de los impuestos, tasas y arbitrios que sobre la vivienda objeto del contrato recaigan o puedan recaer en el futuro, así como los gastos de los demás servicios no susceptibles de individualización, ya que estos últimos correrán en todo caso por cuenta del arrendatario.\n\nEn el momento de la formalización del presente contrato, el importe de dichos gastos es el siguiente:\n\n- Impuesto sobre Bienes Inmuebles: ........... euros anuales.\n\n- Cuota mensual de Comunidad (ej.):  ........... euros.\n\n- Tasa de vado: .................euros anuales.\n\nEl arrendatario se obliga asimismo a abonar las elevaciones que en estos gastos se produzcan. Durante los cinco primeros años del contrato, a las subidas de los gastos distintos de los impuestos se aplicará el límite máximo establecido del doble del IPC General.\n\nNota 4. La Ley permite utilizar otras referencias para la actualización, pero siempre con el límite máximo de la variación anual del IPC. Le proponemos el IPC general por ser objetivo, oficial y sujeto, en principio, a variaciones moderadas. Si no se pacta un sistema de actualización, NO se podrá actualizar la renta durante la vigencia del contrato.\n\nNota 5. O la fecha de ocupación si fue posterior.\n\nNota 6. En el modelo sólo se reseñan a modo de ejemplo algunas posibles prohibiciones u obligaciones. Deben consignarse todas las prohibiciones que el arrendador desee imponer al arrendatario. De no hacerse así, solo serán causas de resolución las expresamente indicadas en la Ley. Para evitar malentendidos conviene ser muy claros en este punto antes de firmar el contrato.\n\nLa Ley de Arrendamientos Urbanos permite que en el contrato las partes acuerden que, durante un plazo determinado, la obligación del pago de la renta se reemplace total o parcialmente por el compromiso del arrendatario de reformar o rehabilitar el inmueble en los términos y condiciones que se pacten y que conviene en todo caso detallar bien. Al finalizar el arrendamiento, el arrendatario no podrá pedir en ningún caso compensación adicional por el coste de las obras realizadas en el inmueble. El incumplimiento por parte del arrendatario de la realización de las obras en los términos y condiciones pactadas podrá ser causa de resolución del contrato de arrendamiento.\n\nNota 7. Si así lo desea, el consentimiento, necesariamente escrito, favorable a la cesión o subarriendo de la vivienda por el arrendatario puede otorgarse a través del propio contrato de arrendamiento. En ese caso, debería incluirse una cláusula del estilo de la siguiente:\n\nEl arrendador otorga su expreso consentimiento para que el arrendador pueda (ceder) subarrendar de forma parcial la vivienda a las personas que considere conveniente. En cualquier caso, el precio del subarriendo no podrá exceder del que corresponda al arrendamiento.\n\nNota 8. Consideramos conveniente hacer un inventario de todos los bienes de la vivienda, lo cual beneficiará al finalizar el arrendamiento, en el caso de posibles problemas, tanto al arrendador como al arrendatario. Deben firmar también todas las partes el inventario.\n\nNota 9. Existen otros tipos de garantías a favor del propietario que pueden incluirse, pero en contratos de arrendamiento de vivienda de hasta cinco años de duración, la Ley prohíbe que el valor de esa garantía adicional exceda de dos mensualidades de renta.\n\nNota 10. Existen otros tipos de garantías a favor del propietario que pueden incluirse, pero en contratos de arrendamiento de vivienda de hasta cinco años de duración, la Ley prohíbe que el valor de esa garantía adicional exceda de dos mensualidades de renta.\n\nHay quien opta por exigir un aval personal (de una persona que sea propietaria de un inmueble identificado), en términos como los siguientes\n\nCLÁUSULA ADICIONAL DE FIANZA SOLIDARIA. - El/los abajo firmante/s afianza/n de modo personal y solidario todas las obligaciones y responsabilidades sin excepción alguna contraídas por el arrendatario en virtud del presente contrato, tanto en las derivadas del plazo inicial del contrato como de sus posibles prórrogas, con renuncia expresa a los beneficios de orden, excusión y división.\n\nD............................. D.N.I. nº........................ C/.......................................................\n\nINVENTARIO QUE ACOMPAÑA AL CONTRATO DE ARRENDAMIENTO\nde la vivienda sita en ......................., calle ....................... núm. ........... piso ........ letra ........., firmado el ....................... de ....................... de .............., entre D/Dña. ......................., como arrendador y D/Dña. ......................., como arrendatario.\n\nEN LA COCINA HAY:\n\n                    5 sartenes\n\n                    3 cacerolas\n\n                    1 olla a presión\n\n                   \n                    1 freidora\n\n        ………. (otros)\n\n                    1 frigorífico marca …\n\n                    1 calentador …\n\n                    1 lavadora marca\n\n                    1 cocina de gas butano\n\n                    ………. (otros)\n\n         \n\nEN EL SALÓN-COMEDOR HAY:\n\n                    2 butacas\n\n                    1 sofá-cama\n\n                    1 mesa\n\n                    6 sillas\n\n                    1 mesa baja\n\n                    1 mueble bar-biblioteca\n\n                    1 televisor en color marca …\n\n                    ………. (otros)\n\n\nEN EL DORMITORIO PRINCIPAL HAY:\n\n                    1 cama de matrimonio\n\n                    2 mesillas de noche\n\n                    1 armario de tres cuerpos\n\n                    1 sillón\n\n                    2 sillas\n\n                    1 cómoda con espejo\n\n                    ………. (otros)\n\nY en prueba de conformidad firman el presente a un solo efecto en el lugar y fecha del contrato anexo.\n\nFirma del arrendador                                                Firma del arrendatario"
      }
    ],
    "denuncias": [
      {
        "title": "Denuncia - Ejemplo 1",
        "content": "ESCRITO DE DENUNCIA \nDatos Generales \nADVERTENCIA\nEN CASO DE DISCONFORMIDAD CON SU DESPIDO, DEBERÁ INTERPONER DEMANDA ANTE LA JURISDICCIÓN SOCIAL EN EL IMPRORROGABLE PLAZO DE 20 DÍAS HÁBILES DESDE LA FECHA DEL DESPIDO.\nEspacio reservado Registro de Entrada\nDirigido a:\n\nEl escrito debe dirigirse a la Inspección Provincial de Trabajo y Seguridad Social correspondiente a la provincia donde radique el centro de trabajo objeto de denuncia.\n\nDatos del Denunciado\n\nNombre o Razón Social:\nNIF / CIF:\n\n\nActividad:\nCCC:\nDomicilio Social:\nCentro Trabajo:\nLocalidad:\nProvincia:\nCódigo Postal:\nNº de Trabajadores\nHorario:\na\n¿Continua abierta la empresa? SI 0\nNO 0\nDatos del Denunciante\nNombre y Apellidos:\nNIF / CIF:\nNAF:\nDomicilio:\nLocalidad:\nCorreo electrónico:\nProvincia:\nCódigo Postal:\nTeléfono:\n¿Es o ha sido trabajador de la empresa? SI 0\nNO 0\nFecha de ingreso:\nFecha de cese:\nMotivo del cese:\nSi actúa en nombre de otros trabajadores indique Nombre y Apellidos:\n¿Tiene presentada demanda judicial por el mismo motivo que esta reclamación?  SI 0\nNO 0\nLa Ley 39/2015, de 1 de octubre del Procedimiento Administrativo Común de las Administraciones Públicas, reconoce a los ciudadanos el derecho a relacionarse con las Administraciones Públicas utilizando medios electrónicos para el ejercicio de los derechos previstos en su art.13.\nDe acuerdo con lo previsto en el artículo 20.4 de la Ley 23/2015 de 21 de julio, el denunciante no podrá alegar la consideración de interesado a ningún efecto en la fase de investigación, y sólo tendrá derecho a recibir información sobre el estado de tramitación de su denuncia, así como de los hechos que se hayan constatado y de las medidas adoptadas al respecto cuando el resultado de la investigación afecte a sus derechos individuales o colectivos reconocidos por la normativa correspondiente al ámbito de la función inspectora.\nEn el supuesto de que la denuncia diera lugar al inicio de un procedimiento sancionador, el denunciante podrá tener, en su caso, la condición de interesado, en los términos y con los requisitos establecidos en el artículo 4 de la Ley 39/2015, de 1 de octubre (B.O.E. de 2 de octubre).\nRELATO DE HECHOS: (Concretar el motivo, acompañando documentación justificativa en su caso)\nFIRMA DEL DENUNCIANTE\nNombre:\nFirmado:  …………………………………………\n\nVoluntad de relacionarse electrónicamente con el OEITSS\nEn principio, dado que usted no es un sujeto obligado a relacionarse electrónicamente con la Administración Pública, recibirá sus notificaciones y comunicaciones en papel. No obstante, puede decidir que la Inspección de Trabajo y Seguridad Social le envíe en lo sucesivo sus comunicaciones y notificaciones exclusivamente de forma electrónica.\nCuando le remitamos una comunicación o una notificación de forma electrónica, le enviaremos un aviso a la dirección de correo electrónico facilitada en este formulario.\nPara acceder a dicha notificación electrónica, será necesario estar dado de alta en Cl@ve o disponer de un certificado electrónico emitido por una autoridad de certificación inscrita en el registro de Prestadores de Servicios de Certificación Electrónica en el Ministerio de Industria. Podrá cambiar el sentido de su decisión en cualquier momento posterior (Arts. 14, 41 y 43 Ley 39/2015, de 1 de octubre del Procedimiento Administrativo Común de las Administraciones Públicas).\nEn base a esto, ¿Confirma que desea seguir recibiendo sus comunicaciones y notificaciones en papel?   Sí, lo confirmo 0\n\nPara aceptar expresamente marque este recuadro con [X].\nEn caso de no marcar este check, recibirá sus comunicaciones y notificaciones exclusivamente de forma electrónica.\nAviso sobre Protección de Datos Personales:\nDe conformidad con lo dispuesto en el artículo 13 del Reglamento (UE) 2016/679 del Parlamento Europeo y del Consejo de 27 de abril de 2016 (Reglamento General de Protección de Datos Personales) y el artículo 11 de la Ley Orgánica 3/2018, de 5 de diciembre, de Protección de Datos Personales y garantía de los derechos digitales, se informa que el presente formulario contiene datos de carácter personal que serán objeto de tratamiento por el responsable, el Organismo Estatal Inspección de Trabajo y Seguridad Social, con la finalidad de desempeñar la función principal de vigilancia y exigencia del cumplimiento de la normativa de orden social y de Seguridad Social, estando para todo ello legitimado en base a lo dispuesto en el artículo 8.2 de la ya citada Ley Orgánica 3/2018, de 5 de diciembre.\nSe podrán ejercer, cuando procedan, los derechos reconocidos en los artículos 15 a 22 del Reglamento (UE) 2016/679, mediante un escrito dirigido al Organismo Estatal Inspección de Trabajo y Seguridad Social, en el Paseo de la Castellana, 63, 28071 Madrid o a través del correo electrónico: pdp.itss@mites.gob.es.\nPodrá accederse a la restante información exigida por el artículo 13 del Reglamento (UE) 2016/679 del Parlamento Europeo y del Consejo de 27 de abril de 2016 (Reglamento General de Protección de Datos Personales) a través de la dirección electrónica que se indica a continuación: https://www.aepd.es/reglamento/derechos/index.html."
      },
      {
        "title": "Denuncia - Ejemplo 2",
        "content": "MODELO DE DENUNCIA PENAL AL JUZGADO DE ....................1ª/2ª, etc. INSTANCIA, INSTRUCCIÓN, GUARDIA de (POBLACIÓN) u otro órgano donde se presente la denuncia. Nombre y apellidos del denunciante (...), mayor de edad, vecino de (...), con domicilio en (...) y con NIF número (...)  comparezco ante este Juzgado (o comisaría de policía u otra autoridad competente) y como mejor proceda en Derecho,  DIGO I. Que por medio del presente formulo DENUNCIA por escrito (si se conociere: contra nombre y apellidos del denunciado y otros datos si se supieran) / y frente a todos aquellos que pudieran resultar responsables tras la instrucción que se practique), por el presunto delito de (...), tipificado en el artículo (...) del Código Penal. (Esta parte puede omitirse si no se sabe exactamente qué legislación se incumple). II. Que los hechos en los que se basa la presente denuncia son los siguientes: (...)  (Descripción de los hechos supuestamente delictivos y las circunstancias en que ocurrieron, lugar, personas involucradas, etc. intentando, dentro de lo posible, ordenar los datos cronológicamente, de la forma más simple). III. Como prueba de estos hechos adjunto al presente escrito las siguientes pruebas (documentos enumerados): (fotografías, vídeos, testimonios, etc.) En su virtud, SUPLICO AL JUZGADO Que tenga por presentado este escrito de denuncia, junto con los documentos que se acompañan,  y se proceda a la averiguación y comprobación de los hechos, para que, tras los trámites legales oportunos, se enjuicie en su día a las personas implicadas en los mismos. Es justicia que solicito, En (...), a (...) de (...) de (...) Firma."
      },
      {
        "title": "Denuncia - Ejemplo 3",
        "content": "Denuncia penal\nAl Juzgado de Instrucción de… que por turno corresponda\n\nD/Dª…, [en su caso, en nombre y representación de…] con domicilio a efectos de notificaciones en…, comparezco y como mejor proceda\n\nDIGO\n\nQue por medio del presente escrito y conforme se dispone en los artículos 259 y siguientes de la Ley de Enjuiciamiento Criminal vengo a formular DENUNCIA contra… por un DELITO CONTRA…, tal y como se detalla a continuación.\nLa presente DENUNCIA  se basa en los siguientes:\n\nHECHOS\n\nI.  Que …\n\nII. …\n\nA los expresados hechos le son de aplicación los siguientes,\n\nFUNDAMENTOS DE DERECHO\n\nPrimero.- Que los citados hechos podrían ser constitutivos del delito  tipificado en el artículo… del Código Penal, el cual dispone que…\n\nSegundo.- Que se propone, a los oportunos efectos acreditativos de los hechos anteriormente narrados, la práctica de las siguientes diligencias probatorias:\n1.  Se tengan por acompañados los documentos UNO a CINCO consistentes en…\n2.  Se oficie al Servicio de Protección de la Naturaleza de la Guardia Civil a los efectos de que informe de…\n3.  Se reciba declaración en calidad de imputado a D./Dª…. con domicilio en…\n4.  Se reciba declaración en calidad de testigo a D/Dª… con domicilio en…\n5.  Aquellas otras que se deriven de las anteriores y sirvan para el esclarecimiento de los hechos.\n\nPor lo expuesto,\nAL JUZGADO SOLICITO que teniendo por presentado este escrito, con sus copias se sirva admitir la presente DENUNCIA contra… acordando la incoación de diligencias previas y la averiguación de los hechos que en el presente escrito se relatan al objeto de esclarecer las eventuales responsabilidades penales dimanantes de los mismos.\n\nOTROSÍ DIGO que, siendo intención de este denunciante mostrarse parte en las Diligencias que se instruyesen a raíz del presente escrito de denuncia como acusación particular o popular, interesa a nuestro derecho y así\nSOLICITO que incoado que sea el oportuno procedimiento penal, se cite a esta parte denunciante a fin de ratificar y en su caso ampliar la presente denuncia y se le realice el correspondiente ofrecimiento de acciones conforme dispone el artículo 776 LECrim.\nLugar, fecha y firma.\nRecurriremos a la denuncia penal en lugar de a la administrativa cuando los hechos que pretendamos denunciar puedan ser constitutivos de delito. Es decir, cuando estén tipificados como tal en el Código Penal.\nLos delitos contra el medio ambiente se regulan en el Título XVI del CP, artículos 319 a 340 y los delitos de incendio en los artículos 351 a 358. Te recomendamos la consulta a la “Guía práctica de los delitos ecológicos”, de la colección Cuadernos prácticos de Ecologistas en Acción.\nTambién podemos dirigir nuestra denuncia a la Guardia Civil (especialmente al Servicio de Protección de la Naturaleza –SEPRONA- de la comarca) o a la Fiscalía (lo que puede ser especialmente interesante allí dónde se hayan creado Fiscalías especializadas en medio ambiente y urbanismo).\nLas indicaciones de la denuncia administrativa son igualmente aplicables en este modelo.\nPor ejemplo, “del delito contra la fauna tipificado en el artículo 334.1 del Código Penal”.\nFotografías, videos, croquis, informes, etc. La documentación oportuna de que dispongamos para acreditar los hechos que denunciamos.\nSi lo conocemos, en caso contrario podemos señalar simplemente “cuyos demás datos personales se desconocen” o que “puede ser citado en … que constituye su lugar de trabajo”, por ejemplo.\nSi la denuncia se realiza al Ministerio Fiscal ha de tenerse en cuenta que no existe la posibilidad de personarse en las Diligencias de Investigación Fiscal que éste incoe, pero sí podemos solicitar que “se nos comunique ‘la resolución que adopte el Fiscal por la que concluya las presentes diligencias de investigación’” a que se refiere el Apartado XI de la Circular 4/2013 de la Fiscalía General del Estado, interesándose asimismo que, judicializada sea la causa, se nos realice el correspondiente ofrecimiento de acciones del artículo 776 LECrim”."
      }
    ]
  }
}

Después de cargar el dataset tiene que ser procesado.

In [3]:
# Preprocesamiento de los datos
def preprocess_data(documents):
    data = []
    for category, items in documents["document_types"].items():
        for item in items:
            data.append({"title": item["title"], "content": item["content"], "category": category})
    return data

# Convertir JSON a un DataFrame
data = preprocess_data(documents)
df = pd.DataFrame(data)

# Convertir categorías a números
category_mapping = {category: idx for idx, category in enumerate(df['category'].unique())}
df['category_id'] = df['category'].map(category_mapping)

# Convertir DataFrame a Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Convertir la columna category_id a ClassLabel
class_label = ClassLabel(num_classes=len(category_mapping), names=list(category_mapping.keys()))
dataset = dataset.cast_column('category_id', class_label)

# Dividir el dataset en conjuntos de entrenamiento y prueba
dataset = dataset.train_test_split(test_size=0.4, stratify_by_column='category_id')

Casting the dataset:   0%|          | 0/12 [00:00<?, ? examples/s]

Despues del procesamiento se cargan todos los componentes del modelo MLM y se configuran los parámetros para la quantización y el entrenamiento.

In [4]:
# Tokenización mrm8488/legal-longformer-base-8192-spanish
tokenizer = AutoTokenizer.from_pretrained("mrm8488/legal-longformer-base-8192-spanish")

def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configuración de BitsAndBytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=False,
)

model_name = 'mrm8488/legal-longformer-base-8192-spanish'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     device_map=device_map,
                                                     quantization_config=bnb_config,
                                                     trust_remote_code=True)

original_model = prepare_model_for_kbit_training(original_model)

# Configuración de Lora
config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'dense'],
    bias="none",
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

original_model.gradient_checkpointing_enable()
peft_model = get_peft_model(original_model, config)

# Override the _make_causal_mask function to ensure no overflow occurs
def _make_causal_mask(input_ids_shape, dtype, device, past_key_values_length=0, sliding_window=False):
    bsz, tgt_len = input_ids_shape
    _MASKING_VALUE = -1e+30 if dtype == torch.float32 else -1e+4
    mask = torch.full((tgt_len, tgt_len), _MASKING_VALUE, device=device, dtype=dtype)
    mask_cond = torch.arange(mask.size(-1), device=device)
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    return mask

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results5/',
    warmup_steps=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,#4 a 2
    max_steps=500, # cambio de 1000
    learning_rate=1e-4, # de 2e-4 a 1e-4 entrenamiento más estable y progresivo.
    optim="paged_adamw_8bit",
    logging_steps=25,
    save_strategy="steps",# de 25 a 50 abajo
    save_steps=50,
    eval_strategy="steps",
    eval_steps=50,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="wandb",
    overwrite_output_dir=True,
    group_by_length=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/906k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/553k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

...

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/613M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


Y por último se instancian las métricas que van a ser recogidas durante este entrenamiento y el entrenador.

In [5]:
# Custom metric computation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Filter out -100 values (used for padding)
    valid_labels = labels[labels != -100]
    valid_predictions = predictions[labels != -100]

    accuracy = accuracy_score(valid_labels, valid_predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(valid_labels, valid_predictions, average='weighted')
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Entrenador
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics=compute_metrics,
)

trainer.model._make_causal_mask = _make_causal_mask

max_steps is given, it will override any value given in num_train_epochs


Por ultimo se comienza el entrenamiento y se guarda el modelo una vez entrenado.

In [6]:
trainer.train()

# Guardar el modelo entrenado
model_save_path = "/content/MyDrive/MyDrive/results5/"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,7.337700,6.160692,0.365834,0.413546,0.365834,0.307350
100,4.901400,4.963107,0.457695,0.431476,0.457695,0.415438
150,3.865400,4.197628,0.363417,0.356908,0.363417,0.335097
200,3.075500,3.635458,0.282031,0.274993,0.282031,0.258417
250,2.526900,3.254286,0.238114,0.228234,0.238114,0.217146
300,2.188800,2.983958,0.206688,0.198517,0.206688,0.187065
350,1.966800,2.814061,0.188961,0.177940,0.188961,0.169260
400,1.817500,2.696877,0.176068,0.164351,0.176068,0.157474
450,1.732400,2.640863,0.169621,0.158509,0.169621,0.153282
500,1.691100,2.617134,0.169218,0.158507,0.169218,0.153529


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.chec

('/content/MyDrive/MyDrive/results5/tokenizer_config.json',
 '/content/MyDrive/MyDrive/results5/special_tokens_map.json',
 '/content/MyDrive/MyDrive/results5/vocab.json',
 '/content/MyDrive/MyDrive/results5/merges.txt',
 '/content/MyDrive/MyDrive/results5/added_tokens.json',
 '/content/MyDrive/MyDrive/results5/tokenizer.json')